### TODO:

1. store in feast
2. modify dataset with the following rules
    - create "standard yearly salary"
    - use the salary and pay_period:
        + yearly -> use the mid-point
        + hourly -> 40 hrs per week * number of weeks in a year
        + monthly -> * 12 months
        + bi-weekly -> * 2 * 12 months
        + else -> consider without salary attribute (use for testing phase - predict)
4. create a dataset with the following requirements
    - only all which have description, min, max salary value

In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet("../data/data_train_without_null.parquet")

In [3]:
df.dtypes, df.shape

(job_id                          int64
 company_name                   object
 title                          object
 description                    object
 max_salary                    float64
 pay_period                     object
 location                       object
 company_id                    float64
 views                         float64
 med_salary                    float64
 min_salary                    float64
 formatted_work_type            object
 applies                       float64
 original_listed_time          float64
 remote_allowed                float64
 job_posting_url                object
 application_url                object
 application_type               object
 expiry                        float64
 closed_time                   float64
 formatted_experience_level     object
 skills_desc                    object
 listed_time                   float64
 posting_domain                 object
 sponsored                       int64
 work_type               

In [4]:
df.isna().sum()

job_id                            0
company_name                    455
title                             0
description                       0
max_salary                        0
pay_period                        0
location                          0
company_id                      454
views                           441
med_salary                    29791
min_salary                        0
formatted_work_type               0
applies                       21907
original_listed_time              0
remote_allowed                25474
job_posting_url                   0
application_url               12056
application_type                  0
expiry                            0
closed_time                   29658
formatted_experience_level     6695
skills_desc                   29590
listed_time                       0
posting_domain                13531
sponsored                         0
work_type                         0
currency                          0
compensation_type           

In [9]:
# remove currencies which are not USD
df.query("currency != 'USD'").count()['currency']

15

In [15]:
df = df.query("currency == 'USD'")

In [16]:
df.count()

job_id                        29776
company_name                  29323
title                         29776
description                   29776
max_salary                    29776
pay_period                    29776
location                      29776
company_id                    29324
views                         29336
med_salary                        0
min_salary                    29776
formatted_work_type           29776
applies                        7882
original_listed_time          29776
remote_allowed                 4315
job_posting_url               29776
application_url               17725
application_type              29776
expiry                        29776
closed_time                     132
formatted_experience_level    23084
skills_desc                     201
listed_time                   29776
posting_domain                16251
sponsored                     29776
work_type                     29776
currency                      29776
compensation_type           

In [124]:
df = df.drop(columns=["sponsored"])

In [ ]:
df.query("company_name")

In [117]:
# df['remote_allowed'] = df['remote_allowed'].astype(str).str.strip().str.replace('', 0.0, regex=True)
# df = df[~df['company_name'].isin([None, 'None'])]
# df = df.fillna(0.0)
# df['remote_allowed'] = df['remote_allowed'].fillna(0.0)
# df['remote_allowed'].value_counts(dropna=False)
import numpy as np
df['remote_allowed'] = df['remote_allowed'].replace(0.0, 0)
df['remote_allowed'] = df['remote_allowed'].astype(int)
# df['remote_allowed'] = df['remote_allowed'].fillna(0.0)


In [140]:
df.count()

company_name         29323
title                29323
description          29323
max_salary           29323
pay_period           29323
location             29323
min_salary           29323
remote_allowed       29323
work_type            29323
normalized_salary    29323
dtype: int64

In [119]:
pd.set_option('display.max_colwidth', None)

In [139]:
# df["pay_period"].value_counts()
# df["location"].value_counts()
# df[:100]["normalized_salary"]
df["work_type"].value_counts() # to convert
# df["remote_allowed"].value_counts() # to convert # done
# df[:100]["company_name"] # to convert
# df[:10]["title"] to convert
# df[:2]["description"] to convert - also maybe investigate skill extraction
# df[:100]["formatted_experience_level"] drop - it could be a case for prediction
# df[:10]["title"]
# df["sponsored"].value_counts()

work_type
FULL_TIME     24167
CONTRACT       3203
PART_TIME      1442
TEMPORARY       278
INTERNSHIP      146
OTHER            86
VOLUNTEER         1
Name: count, dtype: int64

- job_id
- title
- description
- max_salary
- pay_period
- med_salary
- min_salary
- compensation_type

In [141]:
df.to_parquet('data_train_features_need_preprocessing.parquet', engine='pyarrow', compression='snappy')

In [16]:
mask = (
    df["title"].notnull() & 
    df["description"].notnull() & 
    df["max_salary"].notnull() & 
    df["pay_period"].notnull() &
    df["min_salary"].notnull()
)

In [17]:
df_train = df[mask].copy()

In [18]:
df_train.dtypes, df_train.shape

(job_id                          int64
 company_name                   object
 title                          object
 description                    object
 max_salary                    float64
 pay_period                     object
 location                       object
 company_id                    float64
 views                         float64
 med_salary                    float64
 min_salary                    float64
 formatted_work_type            object
 applies                       float64
 original_listed_time          float64
 remote_allowed                float64
 job_posting_url                object
 application_url                object
 application_type               object
 expiry                        float64
 closed_time                   float64
 formatted_experience_level     object
 skills_desc                    object
 listed_time                   float64
 posting_domain                 object
 sponsored                       int64
 work_type               

In [19]:
df_predict = df[~mask].copy()

In [20]:
df_predict.dtypes, df_predict.shape

(job_id                          int64
 company_name                   object
 title                          object
 description                    object
 max_salary                    float64
 pay_period                     object
 location                       object
 company_id                    float64
 views                         float64
 med_salary                    float64
 min_salary                    float64
 formatted_work_type            object
 applies                       float64
 original_listed_time          float64
 remote_allowed                float64
 job_posting_url                object
 application_url                object
 application_type               object
 expiry                        float64
 closed_time                   float64
 formatted_experience_level     object
 skills_desc                    object
 listed_time                   float64
 posting_domain                 object
 sponsored                       int64
 work_type               

In [21]:
assert (df_train.shape[0] + df_predict.shape[0]) == df.shape[0], "not correct data size, possibly data are missed"